<a href="https://colab.research.google.com/github/yxbxn/NLP_contest/blob/main/KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/MyDrive')

In [ ]:
# 특정 파일 불러오기
import pandas as pd
train_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/통계청 인공지능 공모전/train_preprocessed_dup.csv")
test_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/통계청 인공지능 공모전/test_preprocessed.csv",index_col = 0)

In [ ]:
train_preprocessed

In [ ]:
test_preprocessed

tsv로 변환
- NaN 채우기
- tst 파일로 변환

In [ ]:
# Dataset 변환 위해 test label NaN 채우기
test_preprocessed["label"] = 0

In [ ]:
test_preprocessed.to_csv("/content/MyDrive/MyDrive/통계청 인공지능 공모전/test_preprocessed.tsv",encoding="utf-8-sig",index=False,sep="\t")
train_preprocessed.to_csv("/content/MyDrive/MyDrive/통계청 인공지능 공모전/train_preprocessed.tsv",encoding="utf-8-sig",index=False,sep="\t")

label 너무 적은 것들 제외하기
- 20미만 제외
- except_label : 해당 label들은 제외하기 개수가 20개 미만

In [ ]:
train_preprocessed["label"].value_counts().describe()

In [ ]:
except_label = train_preprocessed["label"].value_counts()[train_preprocessed["label"].value_counts() < 2].index
except_label

2개 미만인 label 제거

In [ ]:
mask = train_preprocessed['label'].isin(except_label)
train_preprocessed = train_preprocessed[~mask].reset_index(drop=True)
train_preprocessed

KoBERT 모델의 입력형식에 맞게 데이터 형식 변환하기
- label을 인코딩하기

In [ ]:
train_preprocessed.head(3)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_preprocessed["label"])

In [ ]:
# 인코딩 정보 기반 transform : https://pinkwink.kr/1247
label = le.transform(train_preprocessed["label"])
train_preprocessed["label"] = label

label 개수

In [ ]:
print("라벨개수",len(train_preprocessed["label"].value_counts()))
train_preprocessed["label"].value_counts()

텍스트 데이터와 label 데이터 담기

In [ ]:
# dataframe 형태의 데이터를 리스트 형태로 변경
# bert 모델을 위한 데이터
def transform_df_lst(df):
    data_lst = []
    for q, label in zip(df['document'], df['label'])  :
        data = []
        data.append(q)
        data.append(str(label))

        data_lst.append(data)
    return data_lst

In [ ]:
train_preprocessed = train_preprocessed[['document','label']]

학습, 평가 데이터 20% 비율로 나누기
- stratify 추가

In [ ]:
#train & test 데이터로 나누기 동일 비율
from sklearn.model_selection import train_test_split                                                   
dataset_train, dataset_test = train_test_split(train_preprocessed, stratify = train_preprocessed["label"]
                                               , test_size=0.20, random_state=0,shuffle=True)

In [ ]:
dataset_train = transform_df_lst(dataset_train)
dataset_test = transform_df_lst(dataset_test)

In [ ]:
print("학습 데이터 개수",len(dataset_train))

KoBERT의 입력데이터로 만들기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# 파라미터 셋팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

data_train : 토큰화 및 패딩된 학습데이터 80%
data_test : 토큰화 및 패딩된 학습데이터 20%

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0]
# 첫번째 array : 패딩된 시퀀스
# 두번째 array : 길이와 타입
# 세번째 array : 어텐션 마스크 시퀀스

torch 형식의 데이터로 만들기

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

KoBERT 학습모델 만들기
- 클래스 수 조정하기

In [ ]:
len(train_preprocessed['label'].unique())

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=223,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.3).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

데이터 학습하기

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

모델 저장하기

In [ ]:
model_save_name = 'classifier.bert_new'
path = F"/content/MyDrive/MyDrive/통계청 인공지능 공모전/{model_save_name}" 
torch.save(model.state_dict(), path)

모델 불러오기

In [ ]:
model_save_name = 'classifier.bert_new'
path = F"/content/MyDrive/MyDrive/통계청 인공지능 공모전/{model_save_name}"
model.load_state_dict(torch.load(path))

In [ ]:
model

예측하기

In [ ]:
max_len = 64

In [ ]:
a = pd.read_csv('/content/MyDrive/MyDrive/통계청 인공지능 공모전/test_preprocessed.tsv', sep='\t')

In [ ]:
a

field_indices : [학습시킬 데이터의 index, 데이터 레이블의 index]\
num_discard_samples : 데이터 상단에서 제외할 row의 개수 (default =0)


In [ ]:
new_test = nlp.data.TSVDataset('/content/MyDrive/MyDrive/통계청 인공지능 공모전/test_preprocessed.tsv', field_indices=[1,0], num_discard_samples=1)

In [ ]:
test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

In [ ]:
predict_val = []
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
  token_ids = token_ids.long().to(device) 
  segment_ids = segment_ids.long().to(device) 
  valid_length= valid_length 
  out = model(token_ids, valid_length, segment_ids)
  prediction = out.cpu().detach().numpy().argmax()
  predict_val.append(prediction)
  if batch_id % 5000 == 0:
     print(str(batch_id) + "번째 문장의 분류 예측값은" + str(prediction) + "입니다.")

인코딩된 label 값 디코딩하기

In [ ]:
test_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/통계청 인공지능 공모전/test_preprocessed.csv",index_col = 0)

In [ ]:
le.inverse_transform(predict_val)

In [ ]:
predict_df = test_preprocessed.copy()

In [ ]:
# predict_df = test_preprocessed.copy().drop("len",axis=1)
predict_df["label"] = le.inverse_transform(predict_val)
predict_df

예측된 결과물 저장

In [ ]:
predict_df.to_csv("/content/MyDrive/MyDrive/통계청 인공지능 공모전/predict_kobert_0331.csv",encoding="utf-8-sig",index=False)

In [ ]:
predict_df

In [ ]:
submission = pd.read_csv('/content/MyDrive/MyDrive/통계청 인공지능 공모전/2. 모델개발용자료.csv')
b= pd.read_csv('/content/MyDrive/MyDrive/통계청 인공지능 공모전/1. 실습용자료.csv')

In [ ]:
submission

In [ ]:
predict_df

In [ ]:
a['digit_1'] = predict_df['digit_1']
a['digit_2'] = predict_df['digit_2']
a['digit_3'] = predict_df['digit_3']

In [ ]:
submission['label'] = predict_df['label']

In [ ]:
submission['label'] = predict_df['label']
submission['digit_1'] = submission['label'].apply(lambda x: x[0])
submission['digit_2'] = submission['label'].apply(lambda x: x[1:3] if len(x)==6 else x[1])
submission['digit_3'] = submission['label'].apply(lambda x: x[3:] if len(x)==6 else x[-2:])
submission.drop(columns = 'label',axis=1,inplace = True)

In [ ]:
submission.to_csv('/content/MyDrive/MyDrive/통계청 인공지능 공모전/kobert_submission_0331.csv')